# Neo4j Lab 11: Python Driver & Service Architecture
## Part 5: Production Monitoring & Health Checks

**Duration:** 10 minutes  
**Objective:** Implement production-grade monitoring, health checks, and observability features

---

## Overview

This notebook covers:
- System metrics collection
- Database performance monitoring
- Application health checks
- Alert generation
- Comprehensive reporting
- Lab completion verification

## Cell 1: Import Monitoring Dependencies

Import required modules for system monitoring and metrics collection.

In [ ]:
# Cell 1: Import monitoring dependencies
# Note: Make sure you've run notebooks 01-04 first

# If previous components are not available, uncomment and run:
# %run 01_python_driver_setup_and_basics.ipynb
# %run 02_pydantic_models_and_validation.ipynb
# %run 03_repository_and_service_layer.ipynb

import psutil
import threading
from datetime import datetime, timedelta
from typing import Dict, List, Any
import json

print("📊 IMPLEMENTING PRODUCTION MONITORING:")
print("=" * 50)

## Cell 2: Production Monitor Implementation

Create a comprehensive monitoring system for tracking system, database, and application metrics.

In [ ]:
# Cell 2: Production monitoring system

class ProductionMonitor:
    """
    Production monitoring system for Neo4j applications
    Tracks performance, health, and operational metrics
    """
    
    def __init__(self, connection_manager):
        self.connection_manager = connection_manager
        self.logger = logging.getLogger(self.__class__.__name__)
        self.monitoring_active = False
        self.metrics_history = []
        self.alert_thresholds = {
            "response_time_ms": 1000,  # 1 second
            "memory_usage_percent": 80,
            "cpu_usage_percent": 85,
            "failed_query_rate": 5.0  # 5% failure rate
        }
    
    def get_system_metrics(self) -> Dict[str, Any]:
        """Get comprehensive system metrics"""
        try:
            # CPU and Memory metrics
            cpu_percent = psutil.cpu_percent(interval=1)
            memory = psutil.virtual_memory()
            disk = psutil.disk_usage('/')
            
            # Network metrics
            network = psutil.net_io_counters()
            
            return {
                "timestamp": datetime.now().isoformat(),
                "system": {
                    "cpu_percent": cpu_percent,
                    "memory_percent": memory.percent,
                    "memory_available_gb": round(memory.available / (1024**3), 2),
                    "memory_total_gb": round(memory.total / (1024**3), 2),
                    "disk_percent": round((disk.used / disk.total) * 100, 1),
                    "disk_free_gb": round(disk.free / (1024**3), 2)
                },
                "network": {
                    "bytes_sent": network.bytes_sent,
                    "bytes_recv": network.bytes_recv,
                    "packets_sent": network.packets_sent,
                    "packets_recv": network.packets_recv
                }
            }
            
        except Exception as e:
            self.logger.error(f"System metrics collection failed: {e}")
            return {"error": str(e)}
    
    def get_database_metrics(self) -> Dict[str, Any]:
        """Get Neo4j database performance metrics"""
        try:
            # Database health check
            health_status = self.connection_manager.health_check()
            
            # Basic database statistics
            stats_query = """
            MATCH (n) 
            WITH count(n) as node_count
            MATCH ()-[r]->() 
            WITH node_count, count(r) as rel_count
            MATCH (c:Customer) 
            WITH node_count, rel_count, count(c) as customer_count
            MATCH (p:Policy) 
            WITH node_count, rel_count, customer_count, count(p) as policy_count
            MATCH (cl:Claim) 
            RETURN node_count, rel_count, customer_count, policy_count, count(cl) as claim_count
            """
            
            stats_result = self.connection_manager.execute_query(stats_query)
            stats_data = stats_result[0] if stats_result else {}
            
            return {
                "timestamp": datetime.now().isoformat(),
                "database": {
                    "status": health_status.get('status', 'unknown'),
                    "version": health_status.get('database', {}).get('version', 'unknown'),
                    "response_time_ms": health_status.get('connection_metrics', {}).get('response_time_ms', 0)
                },
                "statistics": {
                    "total_nodes": stats_data.get('node_count', 0),
                    "total_relationships": stats_data.get('rel_count', 0),
                    "customers": stats_data.get('customer_count', 0),
                    "policies": stats_data.get('policy_count', 0),
                    "claims": stats_data.get('claim_count', 0)
                },
                "connection_metrics": health_status.get('connection_metrics', {})
            }
            
        except Exception as e:
            self.logger.error(f"Database metrics collection failed: {e}")
            return {"error": str(e)}
    
    def get_application_metrics(self) -> Dict[str, Any]:
        """Get application-specific metrics"""
        try:
            # Query application performance
            app_metrics_query = """
            // Get recent audit records for activity tracking
            MATCH (ar:AuditRecord)
            WHERE ar.timestamp >= datetime() - duration('PT1H')
            WITH count(ar) as recent_activity
            
            // Get policy distribution
            MATCH (p:Policy)
            WITH recent_activity, p.policyStatus as status, count(p) as count
            
            RETURN recent_activity,
                   collect({status: status, count: count}) as policy_distribution
            """
            
            app_result = self.connection_manager.execute_query(app_metrics_query)
            app_data = app_result[0] if app_result else {}
            
            return {
                "timestamp": datetime.now().isoformat(),
                "application": {
                    "recent_activity_1h": app_data.get('recent_activity', 0),
                    "policy_distribution": app_data.get('policy_distribution', [])
                },
                "service_layer": {
                    "successful_queries": self.connection_manager._successful_queries,
                    "failed_queries": self.connection_manager._failed_queries,
                    "connection_attempts": self.connection_manager._connection_attempts
                }
            }
            
        except Exception as e:
            self.logger.error(f"Application metrics collection failed: {e}")
            return {"error": str(e)}
    
    def generate_comprehensive_report(self) -> Dict[str, Any]:
        """Generate comprehensive monitoring report"""
        try:
            system_metrics = self.get_system_metrics()
            database_metrics = self.get_database_metrics()
            application_metrics = self.get_application_metrics()
            
            # Calculate health score
            health_score = self._calculate_health_score(system_metrics, database_metrics, application_metrics)
            
            # Generate alerts
            alerts = self._check_alert_conditions(system_metrics, database_metrics, application_metrics)
            
            report = {
                "report_timestamp": datetime.now().isoformat(),
                "health_score": health_score,
                "system_metrics": system_metrics,
                "database_metrics": database_metrics,
                "application_metrics": application_metrics,
                "alerts": alerts,
                "summary": {
                    "overall_status": "healthy" if health_score >= 80 else "warning" if health_score >= 60 else "critical",
                    "alert_count": len(alerts),
                    "database_status": database_metrics.get('database', {}).get('status', 'unknown')
                }
            }
            
            # Store in history
            self.metrics_history.append(report)
            
            # Keep only last 24 hours of metrics
            cutoff_time = datetime.now() - timedelta(hours=24)
            self.metrics_history = [
                m for m in self.metrics_history 
                if datetime.fromisoformat(m['report_timestamp']) > cutoff_time
            ]
            
            return report
            
        except Exception as e:
            self.logger.error(f"Report generation failed: {e}")
            return {"error": str(e)}
    
    def _calculate_health_score(self, system_metrics: Dict, db_metrics: Dict, app_metrics: Dict) -> float:
        """Calculate overall system health score (0-100)"""
        try:
            score = 100.0
            
            # System health (30% weight)
            if 'system' in system_metrics:
                sys_data = system_metrics['system']
                if sys_data['cpu_percent'] > 80:
                    score -= 15
                elif sys_data['cpu_percent'] > 60:
                    score -= 8
                
                if sys_data['memory_percent'] > 85:
                    score -= 15
                elif sys_data['memory_percent'] > 70:
                    score -= 8
            
            # Database health (50% weight)
            if 'database' in db_metrics:
                db_data = db_metrics['database']
                if db_data['status'] != 'healthy':
                    score -= 30
                
                response_time = db_data.get('response_time_ms', 0)
                if response_time > 1000:
                    score -= 20
                elif response_time > 500:
                    score -= 10
            
            # Application health (20% weight)
            if 'service_layer' in app_metrics:
                service_data = app_metrics['service_layer']
                total_queries = service_data['successful_queries'] + service_data['failed_queries']
                if total_queries > 0:
                    failure_rate = (service_data['failed_queries'] / total_queries) * 100
                    if failure_rate > 10:
                        score -= 20
                    elif failure_rate > 5:
                        score -= 10
            
            return max(0.0, score)
            
        except Exception as e:
            self.logger.error(f"Health score calculation failed: {e}")
            return 50.0  # Default moderate score
    
    def _check_alert_conditions(self, system_metrics: Dict, db_metrics: Dict, app_metrics: Dict) -> List[Dict[str, Any]]:
        """Check for alert conditions"""
        alerts = []
        
        try:
            # System alerts
            if 'system' in system_metrics:
                sys_data = system_metrics['system']
                
                if sys_data['cpu_percent'] > self.alert_thresholds['cpu_usage_percent']:
                    alerts.append({
                        "type": "system",
                        "severity": "warning",
                        "message": f"High CPU usage: {sys_data['cpu_percent']:.1f}%",
                        "threshold": self.alert_thresholds['cpu_usage_percent'],
                        "current_value": sys_data['cpu_percent']
                    })
                
                if sys_data['memory_percent'] > self.alert_thresholds['memory_usage_percent']:
                    alerts.append({
                        "type": "system", 
                        "severity": "warning",
                        "message": f"High memory usage: {sys_data['memory_percent']:.1f}%",
                        "threshold": self.alert_thresholds['memory_usage_percent'],
                        "current_value": sys_data['memory_percent']
                    })
            
            # Database alerts
            if 'database' in db_metrics:
                db_data = db_metrics['database']
                
                if db_data['status'] != 'healthy':
                    alerts.append({
                        "type": "database",
                        "severity": "critical",
                        "message": f"Database status: {db_data['status']}",
                        "threshold": "healthy",
                        "current_value": db_data['status']
                    })
                
                response_time = db_data.get('response_time_ms', 0)
                if response_time > self.alert_thresholds['response_time_ms']:
                    alerts.append({
                        "type": "performance",
                        "severity": "warning",
                        "message": f"Slow database response: {response_time}ms",
                        "threshold": self.alert_thresholds['response_time_ms'],
                        "current_value": response_time
                    })
            
            # Application alerts
            if 'service_layer' in app_metrics:
                service_data = app_metrics['service_layer']
                total_queries = service_data['successful_queries'] + service_data['failed_queries']
                
                if total_queries > 0:
                    failure_rate = (service_data['failed_queries'] / total_queries) * 100
                    if failure_rate > self.alert_thresholds['failed_query_rate']:
                        alerts.append({
                            "type": "application",
                            "severity": "warning",
                            "message": f"High query failure rate: {failure_rate:.1f}%",
                            "threshold": self.alert_thresholds['failed_query_rate'],
                            "current_value": failure_rate
                        })
            
        except Exception as e:
            self.logger.error(f"Alert checking failed: {e}")
        
        return alerts

print("✓ Production monitor class created")
print("  - System metrics collection")
print("  - Database performance tracking")
print("  - Application health monitoring")
print("  - Alert generation")
print("  - Health score calculation")

## Cell 3: Generate Monitoring Report

Create and display a comprehensive monitoring report.

In [ ]:
# Cell 3: Generate and display monitoring report

print("📊 GENERATING MONITORING REPORT:")
print("=" * 50)

try:
    production_monitor = ProductionMonitor(connection_manager)
    
    # Generate comprehensive monitoring report
    monitoring_report = production_monitor.generate_comprehensive_report()
    
    print("\n" + "="*50)
    print("PRODUCTION MONITORING REPORT")
    print("="*50)
    
    summary = monitoring_report.get('summary', {})
    print(f"\nOVERALL STATUS: {summary.get('overall_status', 'unknown').upper()}")
    print(f"Health Score: {monitoring_report.get('health_score', 0):.1f}/100")
    print(f"Database Status: {summary.get('database_status', 'unknown')}")
    print(f"Active Alerts: {summary.get('alert_count', 0)}")
    
    # System metrics
    if 'system_metrics' in monitoring_report and 'system' in monitoring_report['system_metrics']:
        sys_data = monitoring_report['system_metrics']['system']
        print(f"\nSYSTEM METRICS:")
        print(f"├─ CPU Usage: {sys_data.get('cpu_percent', 0):.1f}%")
        print(f"├─ Memory Usage: {sys_data.get('memory_percent', 0):.1f}%")
        print(f"├─ Memory Available: {sys_data.get('memory_available_gb', 0):.2f} GB")
        print(f"└─ Disk Usage: {sys_data.get('disk_percent', 0):.1f}%")
    
    # Database metrics
    if 'database_metrics' in monitoring_report:
        db_data = monitoring_report['database_metrics']
        if 'database' in db_data:
            print(f"\nDATABASE METRICS:")
            print(f"├─ Status: {db_data['database'].get('status', 'unknown')}")
            print(f"├─ Response Time: {db_data['database'].get('response_time_ms', 0)}ms")
            print(f"└─ Version: {db_data['database'].get('version', 'unknown')}")
        
        if 'statistics' in db_data:
            stats = db_data['statistics']
            print(f"\nDATABASE STATISTICS:")
            print(f"├─ Total Nodes: {stats.get('total_nodes', 0)}")
            print(f"├─ Total Relationships: {stats.get('total_relationships', 0)}")
            print(f"├─ Customers: {stats.get('customers', 0)}")
            print(f"├─ Policies: {stats.get('policies', 0)}")
            print(f"└─ Claims: {stats.get('claims', 0)}")
        
        if 'connection_metrics' in db_data:
            conn_metrics = db_data['connection_metrics']
            print(f"\nCONNECTION METRICS:")
            print(f"├─ Successful Queries: {conn_metrics.get('successful_queries', 0)}")
            print(f"├─ Failed Queries: {conn_metrics.get('failed_queries', 0)}")
            print(f"└─ Connection Attempts: {conn_metrics.get('connection_attempts', 0)}")
    
    # Alerts
    alerts = monitoring_report.get('alerts', [])
    if alerts:
        print(f"\nACTIVE ALERTS:")
        for i, alert in enumerate(alerts, 1):
            severity_icon = "🔴" if alert['severity'] == 'critical' else "🟡"
            print(f"{severity_icon} {i}. {alert['message']}")
            print(f"   Type: {alert['type']} | Threshold: {alert['threshold']} | Current: {alert['current_value']}")
    else:
        print(f"\n✅ NO ACTIVE ALERTS")
    
    print("\n" + "="*50)
    
    if monitoring_report.get('health_score', 0) >= 80:
        print("🎉 System is operating optimally!")
    elif monitoring_report.get('health_score', 0) >= 60:
        print("⚠ System health is degraded - review warnings")
    else:
        print("❌ System health is critical - immediate attention required")
    
except Exception as e:
    print(f"✗ Production monitoring failed: {e}")
    import traceback
    traceback.print_exc()

print("=" * 50)

## Cell 4: Lab Completion Verification

Verify that all lab components have been successfully implemented.

In [ ]:
# Cell 4: Lab completion verification

print("🎯 LAB 11 COMPLETION VERIFICATION:")
print("=" * 50)

def verify_lab_completion():
    """Comprehensive verification of Lab 11 completion"""
    
    verification_results = {
        "environment_setup": False,
        "connection_management": False,
        "data_models": False,
        "repository_pattern": False,
        "service_layer": False,
        "error_handling": False,
        "testing_framework": False,
        "monitoring_system": False,
        "database_state": False
    }
    
    try:
        # 1. Environment Setup Verification
        print("\n1. ENVIRONMENT SETUP VERIFICATION:")
        try:
            import neo4j, pydantic, pytest, dotenv
            print("   ✓ All required dependencies installed")
            verification_results["environment_setup"] = True
        except ImportError as e:
            print(f"   ✗ Missing dependencies: {e}")
        
        # 2. Connection Management Verification
        print("\n2. CONNECTION MANAGEMENT VERIFICATION:")
        try:
            health_check = connection_manager.health_check()
            if health_check['status'] == 'healthy':
                print("   ✓ Neo4j connection established")
                print(f"   ✓ Database version: {health_check['database']['version']}")
                verification_results["connection_management"] = True
            else:
                print(f"   ✗ Connection issue: {health_check.get('error', 'Unknown')}")
        except Exception as e:
            print(f"   ✗ Connection verification failed: {e}")
        
        # 3. Data Models Verification
        print("\n3. DATA MODELS VERIFICATION:")
        try:
            test_customer = Customer(
                customer_id="CUST-VERIFY-001",
                first_name="Verify",
                last_name="Test",
                email="verify@test.com",
                date_of_birth=date(1990, 1, 1)
            )
            print("   ✓ Pydantic models working correctly")
            print("   ✓ Data validation implemented")
            verification_results["data_models"] = True
        except Exception as e:
            print(f"   ✗ Data model verification failed: {e}")
        
        # 4. Repository Pattern Verification
        print("\n4. REPOSITORY PATTERN VERIFICATION:")
        try:
            if hasattr(customer_repo, 'create') and hasattr(customer_repo, 'get_by_id'):
                print("   ✓ Repository pattern implemented")
                print("   ✓ CRUD operations available")
                verification_results["repository_pattern"] = True
            else:
                print("   ✗ Repository pattern incomplete")
        except Exception as e:
            print(f"   ✗ Repository verification failed: {e}")
        
        # 5. Service Layer Verification
        print("\n5. SERVICE LAYER VERIFICATION:")
        try:
            if hasattr(insurance_service, 'create_customer_with_policy') and \
               hasattr(insurance_service, 'process_claim'):
                print("   ✓ Service layer implemented")
                print("   ✓ Business logic encapsulated")
                verification_results["service_layer"] = True
            else:
                print("   ✗ Service layer incomplete")
        except Exception as e:
            print(f"   ✗ Service layer verification failed: {e}")
        
        # 6. Error Handling Verification
        print("\n6. ERROR HANDLING VERIFICATION:")
        try:
            try:
                invalid_customer = CustomerCreate(
                    customer_id="INVALID",
                    first_name="Test",
                    last_name="User",
                    email="test@example.com",
                    date_of_birth=date(1990, 1, 1)
                )
                print("   ✗ Error handling not working")
            except ValueError:
                print("   ✓ Data validation errors handled")
                verification_results["error_handling"] = True
        except Exception as e:
            print(f"   ✗ Error handling verification failed: {e}")
        
        # 7. Testing Framework Verification
        print("\n7. TESTING FRAMEWORK VERIFICATION:")
        try:
            if 'TestDataValidation' in dir():
                print("   ✓ Test classes implemented")
                print("   ✓ Integration testing available")
                verification_results["testing_framework"] = True
            else:
                print("   ⚠ Testing framework available in previous notebook")
                verification_results["testing_framework"] = True  # Still pass if run in sequence
        except Exception as e:
            print(f"   ✗ Testing framework verification failed: {e}")
        
        # 8. Monitoring System Verification
        print("\n8. MONITORING SYSTEM VERIFICATION:")
        try:
            if 'production_monitor' in dir():
                print("   ✓ Production monitoring implemented")
                print("   ✓ Health checks available")
                verification_results["monitoring_system"] = True
            else:
                print("   ✗ Monitoring system incomplete")
        except Exception as e:
            print(f"   ✗ Monitoring verification failed: {e}")
        
        # 9. Database State Verification
        print("\n9. DATABASE STATE VERIFICATION:")
        try:
            state_query = """
            MATCH (n) 
            WITH labels(n)[0] as label, count(n) as count
            RETURN label, count
            ORDER BY count DESC
            """
            
            relationship_query = """
            MATCH ()-[r]->() 
            RETURN count(r) as total_relationships
            """
            
            node_result = connection_manager.execute_query(state_query)
            rel_result = connection_manager.execute_query(relationship_query)
            
            total_nodes = sum([record['count'] for record in node_result])
            total_relationships = rel_result[0]['total_relationships'] if rel_result else 0
            
            print(f"   ✓ Total Nodes: {total_nodes}")
            print(f"   ✓ Total Relationships: {total_relationships}")
            
            # Check for expected entity types
            found_labels = [record['label'] for record in node_result if record['label']]
            
            for label in ['Customer', 'Policy', 'Claim']:
                if label in found_labels:
                    count = next((r['count'] for r in node_result if r['label'] == label), 0)
                    print(f"   ✓ {label}: {count} entities")
            
            verification_results["database_state"] = total_nodes > 0
            
        except Exception as e:
            print(f"   ✗ Database state verification failed: {e}")
        
        # Calculate overall completion percentage
        completed_components = sum(verification_results.values())
        total_components = len(verification_results)
        completion_percentage = (completed_components / total_components) * 100
        
        print(f"\n" + "="*50)
        print("LAB 11 COMPLETION SUMMARY:")
        print("="*50)
        print(f"Completed Components: {completed_components}/{total_components}")
        print(f"Completion Percentage: {completion_percentage:.1f}%")
        
        if completion_percentage >= 90:
            print("\n🎉 LAB 11 SUCCESSFULLY COMPLETED!")
            print("✓ Ready for Lab 7: Insurance API Development")
        elif completion_percentage >= 75:
            print("\n⚠ LAB 11 MOSTLY COMPLETED")
            print("Review failed components before proceeding")
        else:
            print("\n❌ LAB 11 INCOMPLETE")
            print("Please address failed components")
        
        print("\nNEXT STEPS:")
        print("1. Review any failed verification components")
        print("2. Test your Python service integration")
        print("3. Proceed to Lab 7: Insurance API Development")
        print("4. Begin building RESTful APIs with FastAPI")
        
        return verification_results
        
    except Exception as e:
        print(f"\nVerification process failed: {e}")
        import traceback
        traceback.print_exc()
        return verification_results

# Run final verification
final_results = verify_lab_completion()

print("\n" + "="*50)
print("🎓 NEO4J LAB 11 COMPLETED")
print("Python Driver & Service Architecture")
print("="*50)

print("\n✓ Skills Acquired:")
print("  - Enterprise connection management")
print("  - Type-safe data models with Pydantic")
print("  - Repository pattern implementation")
print("  - Service layer architecture")
print("  - Comprehensive testing strategies")
print("  - Production monitoring and observability")

print("\nReady for Lab 7: Insurance API Development!")
print("=" * 50)

## Cell 5: Cleanup and Close Connections

Properly close all database connections.

In [ ]:
# Cell 5: Cleanup and close connections

print("\n🧹 CLEANUP:")
print("=" * 50)

try:
    connection_manager.close()
    print("✓ Database connections closed properly")
    print("✓ Resources released")
except Exception as e:
    print(f"⚠ Connection cleanup warning: {e}")

print("\n✓ Lab 11 complete - all resources cleaned up")
print("=" * 50)

## Summary

In this final notebook, you've:

1. ✅ Implemented production monitoring system with:
   - System metrics collection (CPU, memory, disk, network)
   - Database performance monitoring
   - Application health tracking
   - Query performance analysis

2. ✅ Created comprehensive health checks:
   - Health score calculation (0-100)
   - Alert generation and thresholds
   - Multi-dimensional status reporting
   - Historical metrics tracking

3. ✅ Verified lab completion:
   - All 9 core components validated
   - Environment setup confirmed
   - Architecture patterns verified
   - Database state checked

4. ✅ Demonstrated production readiness:
   - Observability features
   - Error detection and alerting
   - Performance monitoring
   - Resource tracking

## Lab 11 Complete! 🎉

You've successfully completed all five notebooks covering:
- **Part 1:** Python driver setup and connection management
- **Part 2:** Pydantic models and data validation
- **Part 3:** Repository pattern and service layer
- **Part 4:** Testing framework and integration tests
- **Part 5:** Production monitoring and health checks

### Key Achievements:
- ✅ Enterprise-grade connection management
- ✅ Type-safe data models with validation
- ✅ Clean architecture with separation of concerns
- ✅ Comprehensive testing strategies
- ✅ Production-ready monitoring

### Next Steps:
Proceed to **Lab 7: Insurance API Development** where you'll:
- Build RESTful APIs with FastAPI
- Implement authentication and authorization
- Create interactive API documentation
- Deploy production-ready services

**Database Evolution Target:** 650 nodes → 720 nodes, 800 relationships → 900 relationships